Demonstrates loading deck.gl as external JSS and CSS components, rendering it as part of an app, and then linking it to Panel widgets.

In [ ]:
import panel as pn

js_files = {'deck': 'https://unpkg.com/deck.gl@^6.0.0/deckgl.min.js',
            'mapboxgl': 'https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.js'}
css_files = ['https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.1/mapbox-gl.css']

pn.extension(js_files=js_files, css_files=css_files)

First, let's declare the cities we are interested in using Python:

In [ ]:
cities = [
    {"city":"San Francisco","state":"California","latitude":37.7751,"longitude":-122.4193},
    {"city":"New York","state":"New York","latitude":40.6643,"longitude":-73.9385},
    {"city":"Los Angeles","state":"California","latitude":34.051597,"longitude":-118.244263},
    {"city":"London","state":"United Kingdom","latitude":51.5074,"longitude":-0.1278},
    {"city":"Hyderabad","state":"India","latitude":17.3850,"longitude":78.4867}]

Next, let's declare an HTML `<div>` to render the plot into, then define the deck.gl script code to render a plot for those cities.

In [ ]:
html = """
<div id="deckgl-container" style="width: 800px; height: 500px;"></div>

<script type="text/javascript">
// Data
var CITIES = %s;
var deckgl = new deck.DeckGL({
  container: 'deckgl-container',
  mapboxApiAccessToken: 'pk.eyJ1IjoicGhpbGlwcGpmciIsImEiOiJjajM2bnE4MWcwMDNxMzNvMHMzcGV3NjlnIn0.976fZ1azCrTh50lEdZTpSg',
  longitude: CITIES[0].longitude,
  latitude: CITIES[0].latitude,
  zoom: 10,
  layers: [
    new deck.ScatterplotLayer({
      data: CITIES,
      getPosition: d => [d.longitude, d.latitude],
      radiusMinPixels: 10
    })
  ]
});
</script>
""" % cities
deckgl = pn.pane.HTML(html, height=500, width=800)

Next we can declare a Panel widget and define a ``jslink`` to update the deck.gl plot whenever the widget state changes. The example is adapted from http://deck.gl/showcases/gallery/viewport-transition but replaces D3 widgets with Panel-based widgets.

In [ ]:
widget = pn.widgets.RadioButtonGroup(options=[c["city"] for c in cities])

update_city = """
var d = CITIES[source.active];
deckgl.setProps({
  viewState: {
    longitude: d.longitude,
    latitude: d.latitude,
    zoom: 10,
    transitionInterpolator: new deck.FlyToInterpolator(),
    transitionDuration: 5000
  }
});
"""

widget.jslink(deckgl, code={'active': update_city});

pn.Column(widget, deckgl)